In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense, Dropout
from keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define the inception module
def inception_module(x):
    conv1 = Conv2D(64, (1,1), padding='same', activation='relu')(x)

    conv3 = Conv2D(64, (1,1), padding='same', activation='relu')(x)
    conv3 = Conv2D(64, (3,3), padding='same', activation='relu')(conv3)

    conv5 = Conv2D(64, (1,1), padding='same', activation='relu')(x)
    conv5 = Conv2D(64, (5,5), padding='same', activation='relu')(conv5)

    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
    pool = Conv2D(64, (1,1), padding='same', activation='relu')(pool)

    output = concatenate([conv1, conv3, conv5, pool], axis=3)

    return output

# Define the model
input_layer = Input(shape=(32, 32, 3))

# Add some convolutional layers before the inception module
x = Conv2D(32, (3,3), padding='same', activation='relu')(input_layer)
x = Conv2D(64, (3,3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Add the inception module
x = inception_module(x)

x = Conv2D(128, (3,3), padding='same', activation='relu')(x)
x = Conv2D(256, (3,3), padding='same', activation='relu')(x)

x = Flatten()(x)
output_layer = Dense(10, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))



Epoch 1/10
782/782 [==============================] - 21s 21ms/step - loss: 1.4049 - accuracy: 0.4942 - val_loss: 1.0835 - val_accuracy: 0.6203
Epoch 2/10
782/782 [==============================] - 15s 20ms/step - loss: 0.9385 - accuracy: 0.6730 - val_loss: 0.8429 - val_accuracy: 0.7100
Epoch 3/10
782/782 [==============================] - 15s 19ms/step - loss: 0.7462 - accuracy: 0.7417 - val_loss: 0.7941 - val_accuracy: 0.7257
Epoch 4/10
782/782 [==============================] - 16s 20ms/step - loss: 0.6190 - accuracy: 0.7852 - val_loss: 0.7467 - val_accuracy: 0.7384
Epoch 5/10
782/782 [==============================] - 16s 20ms/step - loss: 0.5025 - accuracy: 0.8243 - val_loss: 0.7238 - val_accuracy: 0.7519
Epoch 6/10
782/782 [==============================] - 15s 20ms/step - loss: 0.3944 - accuracy: 0.8629 - val_loss: 0.7950 - val_accuracy: 0.7503
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3041 - accuracy: 0.8925 - val_loss: 0.8978 - val_accuracy: